In [1]:
import numpy as np
import random

In [2]:
# A simple RL sequence recommendation model. There are 3 user states {-1, 0, 1} and two possible items {-1,1}
# A sequence is recommended to the user, who considers each item in turn and either skips it or consumes it

# After each item interaction, the system may transition to a new state, at which point a 
# new sequence is recommended to the user. The system must transition after N steps, if a transition has 
# not occurred earlier.

# A recommendation is an N-length slate of items, such as [-1,-1,1,1] when N=4


In [3]:
# slate size

N = 5

# state space size

S = 3

In [4]:
# transition probability

# this is the probability that the system transitions to a new user state after interacting with
# an item

ps = 0.3

In [5]:
# choose which skip model to use, as described in next cell
skipmodel = 2

In [6]:
# skip probabilities

# here we define a model of the probability that an item will be skipped
# depending on the history of items that have been interacted with so far

# 

if skipmodel ==1:

 pskip = 0.1; # probability of a skip when state=0
 pskippospos = 0.0; # probability of a skip when state=1 and item=1
 pskipnegneg = 0.0; # probability of a skip when state=-1 and item=-1
 pskipposneg = 0.2; # probability of a skip when state=1 and item=-1
 pskipnegpos = 0.2; # probability of a skip when state=-1 and item=1
 
if skipmodel == 2:
 pskip = 0.1; # probability of a skip when state=0
 pskippospos = 0.0; # probability of a skip when state=1 and item=1
 pskipnegneg = 0.1; # probability of a skip when state=-1 and item=-1
 pskipposneg = 0.2; # probability of a skip when state=1 and item=-1
 pskipnegpos = 0.2; # probability of a skip when state=-1 and item=1




# adjust these probabilities, depending on a history of two past items
# skipProb[state, item, prev_item, prev_prev_item]
# (This is just completely arbitrary - no basis for the formulae used below)


skipProb = np.zeros([3, 2, 2, 2])


skipProb[0,0,0,0] = 1-(1-pskipnegneg)**2
skipProb[0,0,0,1] = 1-(1-pskipnegneg)
skipProb[0,0,1,0] = 1-(1-pskipnegneg)**0.5
skipProb[0,0,1,1] = 1-(1-pskipnegneg)**0.25
skipProb[0,1,0,0] = 1-(1-pskipnegpos)**0.25
skipProb[0,1,0,1] = 1-(1-pskipnegpos)**0.5
skipProb[0,1,1,0] = 1-(1-pskipnegpos)
skipProb[0,1,1,1] = 1-(1-pskipnegpos)**2
skipProb[1,0,0,0] = 1-(1-pskip)**2
skipProb[1,0,0,1] = 1-(1-pskip)
skipProb[1,0,1,0] = 1-(1-pskip)**0.5
skipProb[1,0,1,1] = 1-(1-pskip)**0.25
skipProb[1,1,0,0] = 1-(1-pskip)**0.25
skipProb[1,1,0,1] = 1-(1-pskip)**0.5
skipProb[1,1,1,0] = 1-(1-pskip)
skipProb[1,1,1,1] = 1-(1-pskip)**2
skipProb[2,0,0,0] = 1-(1-pskipposneg)**2
skipProb[2,0,0,1] = 1-(1-pskipposneg)
skipProb[2,0,1,0] = 1-(1-pskipposneg)**0.5
skipProb[2,0,1,1] = 1-(1-pskipposneg)**0.25
skipProb[2,1,0,0] = 1-(1-pskippospos)**0.25
skipProb[2,1,0,1] = 1-(1-pskippospos)**0.5
skipProb[2,1,1,0] = 1-(1-pskippospos)
skipProb[2,1,1,1] = 1-(1-pskippospos)**2


In [7]:
# map a slate to an index in the range [0,2**N)

def indexOfSlate(slate):
    slate = (slate+1)/2
    N = len(slate)
    indx = 0
    for k in range(N):
        indx = indx + slate[k]*2**(N-k-1)

    return int(indx)
 

In [8]:
indexOfSlate(np.array([-1,1,1,1,1]))

15

In [9]:
# map an index in the range [0,2**N) back to the corresponding slate
def slateOfIndex(indx,N):

    slate = []
    
    for i in range(N):
        slate = [indx%2] + slate
        indx = indx//2
    slate = np.array(slate)
    slate = 2*slate-1
    return slate.astype(int)

In [10]:
slateOfIndex(4,5)

array([-1, -1,  1, -1, -1])

In [121]:
# map an index in the range [0,num_states) to the corresponding state

def stateOfIndex(indx):
    state = indx - 1
    return state

In [122]:
# map a state to an index in the range [0,num_states). 

def indexOfState(state):
    indx = state + 1
    return indx

# User state generation functions

In [123]:

def randState():
 return 2*round(random.random())-1


# Slate Generation functions

In [124]:
def randSlate(N,s=0,otherargs=[]):
    slate = 2*np.round(np.random.random(N))-1
    return slate.astype(int)

In [125]:
randSlate(5)

array([ 1,  1, -1,  1,  1])

In [126]:
def posSlate(N,s=0,otherargs=[]):
    slate = np.ones(N).astype(int)
    return slate

In [127]:
def negSlate(N,s=0,otherargs=[]):
    slate = -np.ones(N).astype(int)
    return slate

In [128]:
def qLearned(N,s=0,qtable=[]):
    st = indexOfState(s)
    action_indx = np.argmax(qtable[st,:])
    slate = slateOfIndex(action_indx,N)
    return slate

# Reward functions

In [129]:
# reward = 1 if the state == item (action) and 0 otherwise
def oddsReward(s, action,k,skip):
    if (~skip and s == action[k]):
        reward = 1
    else:
        reward = 0
 
    return reward


# Transition functions

In [130]:
# If an item is consumed, then if state != 0, state = 0, otherwise state = consumed item
# Idea is that user has consumed an item, then is not intereted to consume again on next round

# Want an interesting transition function that yields long term value ... experiment more here

def doTransition(s, slate, k, skip):
    state = s
    if (~skip):
        if (s==slate[k] or s == -slate[k]):
            state = 0

    if (s==0):
        state = slate[k]
    
    return state

# Skip functions

In [131]:
# Determine whether or not to skip. Use the skipProb model - if haven't sufficient items in the history, then
# average over the missing history slots

def doSkip(s, slate, k):
   
   s_indx = indexOfState(s)
   slate_index = ((slate+1)/2).astype(int)

   r = random.random()
   
   if (k==0):
      testp = np.mean(np.mean(skipProb[s_indx,slate_index[k],:,:]))
   elif (k==1):
      testp = np.mean(skipProb[s_indx,slate_index[k],slate_index[k-1],:])
   else:
      testp = skipProb[s_indx,slate_index[k],slate_index[k-1],slate_index[k-2]]
   
   
   skip = r < testp
      
   return skip, testp


In [132]:
doSkip(-1,np.array([1,1,1,1,1]),1)

(False, 0.2799999999999999)

# Game

In [133]:
# play the game, returning the cumulative reward over T state transitions. Also, return the 
# training data, consisting of the (s, a, r, s') tuples required to train a q-learning algorithm

# Arguements:

# serveSlate = function used to generate a slate
# T = number of transitions
# N = size of slate
# qtable = a table of q-values, which is used when using a Q-table to select the next slate
# randomState = function to generate a random state
# rewardFunction = function to generate the reward
# transitionFunction = function to select the next state to transition to
# skipFunction = function to determine if an item is skipped or not


def game(serveSlate=randSlate,T=1000, N=3, qtable=np.zeros([3,2**N]),
         randomState=randState, rewardFunction=oddsReward, transitionFunction=doTransition,
         skipFunction=doSkip):


    randomState = randState
    rewardFunction = oddsReward
    transitionFunction = doTransition
    skipFunction = doSkip

    cumr = 0
    s = randState()

    training = []

    t = 0
    while (t<T):
        slate = serveSlate(N,s,qtable)
    
        slate_reward = 0
        r_sample = np.zeros(N)
        sk_sample = np.zeros(N)
        for k in range(N):
            skip,testp = skipFunction(s, slate,k)
            
            
            r = rewardFunction(s, slate,k,skip)
            r_sample[k] = r
            sk_sample[k] = skip
            slate_reward = slate_reward + r
            if (random.random()<ps):
                break
                
        cumr = cumr + slate_reward
    
    
        next_s = transitionFunction(s,slate,k,skip)
        training.append([indexOfState(s), 
            indexOfSlate(slate), slate_reward, indexOfState(next_s),
            k, sk_sample, r_sample])
        
        s = next_s
        t = t+1
        
    return cumr, training

In [134]:
def evaluatePolicy(policy, Qtable =[],numSamples=100, numTrans=1000):
    c = 0
    for i in range(numSamples):
        cum_r, train = game(policy,numTrans,N,Qtable)
        c = c+cum_r
    
    return c/numSamples
    

# TD Q-learning over slates

In [135]:
def qlearning(N, S, training, alpha, gamma, test=False):

    # N = slate size
    # S = size of state space

    # size of action space
    A = 2**N


    qtable = np.random.random([S, A])

    t = 0
    for sample in training:
    
        curr_s = sample[0]
        action = sample[1]
        reward = sample[2]
        next_s = sample[3]
    
        qtable[curr_s, action] = (1-alpha)*qtable[curr_s,action] + \
            alpha*(reward + gamma*np.max(qtable[next_s,:]))
                        
        if test and (t % 500) == 0 :
            
            v = evaluatePolicy(qLearned,qtable)     
            action =np.argmax(qtable,axis=1)
            print(action, end=' ')
            print(t,'\t', v)

        t = t+1
                    
                    
    return qtable

In [136]:
def slateExpectedValue(s_indx,slate_indx,N,qtable,qtableN):

    slate = slateOfIndex(slate_indx,N)
    s = stateOfIndex(s_indx)
    slate_q_value = 0
    qt = qtable
    for i in range(N):
        if (i==N-1):
            qt = qtableN

        slate_indx = indexOfSlate(np.array([slate[i]]))
        slate_indx_skip = slate_indx + 2
    
        t,skip_prob = doSkip(s, slate,i)
        slate_q_value = slate_q_value + \
            ((1-ps)**(i))*((1-skip_prob)*qt[s_indx,slate_indx] + skip_prob*qt[s_indx,slate_indx_skip])
        
    return slate_q_value

# Itemwise TD Q-learning using Expected value of Slate

In [137]:
def slateQLearning(N, S, training, alpha, gamma,test=False):


    # N = slate size
    # S = size of state space

    # size of action space
    A = 2*2


    Qtable = np.random.random([S, A])
    QtableN = Qtable
    Qslate = np.zeros([S,2**N])

    t = 0
    for sample in training:
    
    
        curr_s = sample[0]
        slate_indx = sample[1]
        slate_reward = sample[2]
        next_s = sample[3]
        trans_pos = sample[4]
        skip_list = sample[5]
        r_list = sample[6]
    
        slate = slateOfIndex(slate_indx,N)
    
        for k in range(trans_pos):
            item = indexOfSlate(np.array([slate[k]]))
            r = r_list[k]
            skip = skip_list[k]
    
            if (skip):
                item = item + 2

        
            for i in range(2**N):
                Qslate[curr_s,i] = slateExpectedValue(curr_s,i,N,Qtable,QtableN)

            Qtable[curr_s, item] = (1-alpha)*Qtable[curr_s,item] + \
                alpha*(r + ps*gamma*np.max(Qslate[curr_s,:]))


    
        k = trans_pos
        item = indexOfSlate(np.array([slate[k]]))
        r = r_list[k]
        skip = skip_list[k]
        if (skip):
            item = item + 2
    
        for i in range(2**N):
            Qslate[next_s,i] = slateExpectedValue(next_s,i,N,Qtable,QtableN)
    
        if k<N-1:
  
            Qtable[curr_s, item] = (1-alpha)*Qtable[curr_s,item] + \
                alpha*(r + ps*gamma*np.max(Qslate[next_s,:]))
        else:
    
            QtableN[curr_s, item] = (1-alpha)*QtableN[curr_s,item] + \
                alpha*(r + gamma*np.max(Qslate[next_s,:]))

    
        if test and (t % 500) == 0 :
            
            v = evaluatePolicy(qLearned,Qtable)     
            action =np.argmax(Qslate,axis=1)
            print(action, end=' ')
            print(t,'\t', v)

        t = t+1
    
    return Qtable, QtableN, Qslate

# Expected value of a Slate in terms of Itemwise Q-values

# Experiments

In [138]:
cum_r, training = game(randSlate,10000)

In [139]:
# quality when we always recommend item = -1

print('Value of Neg Policy', evaluatePolicy(negSlate))

Value of Neg Policy 1282.9


In [140]:
# quality when we always recommend item = 1


print('Value of Pos Policy', evaluatePolicy(posSlate))

Value of Pos Policy 1384.92


In [141]:
qtable = qlearning(N, S, training, 0.01, 0.1,True)

[18  4  6] 0 	 668.58
[18  9  6] 500 	 588.45
[0 9 6] 1000 	 908.24
[0 9 6] 1500 	 905.52
[0 9 7] 2000 	 965.6
[0 9 7] 2500 	 962.98
[0 9 7] 3000 	 972.59
[0 9 7] 3500 	 967.65
[0 9 7] 4000 	 970.42
[0 9 7] 4500 	 973.91
[0 9 7] 5000 	 964.81
[0 9 7] 5500 	 961.75
[0 9 7] 6000 	 970.61
[0 9 7] 6500 	 969.27
[0 9 7] 7000 	 968.4
[0 9 7] 7500 	 963.22
[0 9 7] 8000 	 964.41
[0 9 7] 8500 	 962.46
[0 9 7] 9000 	 968.13
[0 9 7] 9500 	 968.45


In [142]:
Qtable, QtableN, Qslate = slateQLearning(N,S,training,0.01,0.1,True)

[ 0 31  0] 0 	 816.46
[31 31  0] 500 	 817.61
[31 31  0] 1000 	 760.81
[31 31  0] 1500 	 758.25
[31 31  0] 2000 	 644.77
[31 31 22] 2500 	 727.9
[31 31 31] 3000 	 803.09
[31 31 22] 3500 	 733.59
[22 31 27] 4000 	 823.16
[31 31 31] 4500 	 728.6
[31 31 31] 5000 	 728.33
[27 31 22] 5500 	 722.06
[27 31 22] 6000 	 733.46
[21 31 22] 6500 	 812.94
[27 31 22] 7000 	 727.69
[21 31 22] 7500 	 813.97
[22 31 22] 8000 	 725.94
[10 31 31] 8500 	 810.87
[10 31 22] 9000 	 815.75
[10 31 22] 9500 	 814.09
